# Mais sobre `SELECT`

- temporary tables
- views
- variables
- subqueries


In [29]:
import mysql.connector
from functools import partial

def get_connection_helper():
    def run_db_query(connection, query, args=None):
        with connection.cursor() as cursor:
            print('Executando query:')
            cursor.execute(query, args)
            for result in cursor:
                print(result)

    connection = mysql.connector.connect(
        host='127.0.0.1',
        port=33062,
        user='megadados',
        password='megadados2020',
        database='sakila'
    )
    return connection, partial(run_db_query, connection)

connection, db = get_connection_helper()

## Aquecimento

Quanta receita foi gerada para cada categoria de filmes? Liste do maior para o menor.

In [30]:
# db("""
# SELECT
#     category.name, SUM(payment.amount) AS total
# FROM
#     payment
#     INNER JOIN rental USING(rental_id)
#     INNER JOIN inventory USING(inventory_id)
#     INNER JOIN film_category USING(film_id)
#     INNER JOIN category USING(category_id)
# GROUP BY
#     category_id
# ORDER BY
#     total DESC
# """)

In [31]:
db("""
SELECT *
FROM
    sales_by_film_category
""")

Executando query:
('Sports', Decimal('5314.21'))
('Sci-Fi', Decimal('4756.98'))
('Animation', Decimal('4656.30'))
('Drama', Decimal('4587.39'))
('Comedy', Decimal('4383.58'))
('Action', Decimal('4375.85'))
('New', Decimal('4351.62'))
('Games', Decimal('4281.33'))
('Foreign', Decimal('4270.67'))
('Family', Decimal('4226.07'))
('Documentary', Decimal('4217.52'))
('Horror', Decimal('3722.54'))
('Children', Decimal('3655.55'))
('Classics', Decimal('3639.59'))
('Travel', Decimal('3549.64'))
('Music', Decimal('3417.72'))


Liste os filmes e o numero de vezes em que foram alugados

In [32]:
db("""
SELECT
    title, COUNT(rental_id) AS cnt
FROM
    film
    LEFT OUTER JOIN inventory USING(film_id)
    LEFT OUTER JOIN rental USING(inventory_id)
GROUP BY
    film_id
ORDER BY
    cnt DESC
LIMIT 10
""")

Executando query:
('BUCKET BROTHERHOOD', 34)
('ROCKETEER MOTHER', 33)
('RIDGEMONT SUBMARINE', 32)
('GRIT CLOCKWORK', 32)
('SCALAWAG DUCK', 32)
('JUGGLER HARDLY', 32)
('FORWARD TEMPLE', 32)
('HOBBIT ALIEN', 31)
('ROBBERS JOON', 31)
('ZORRO ARK', 31)


## Views

Uma *view* é uma tabela virtual, construida a partir de um comando `SELECT`. Por exemplo: execute o código a seguir.

In [33]:
db('''
DROP VIEW IF EXISTS movie_count;
''')

db('''
CREATE VIEW movie_count AS
    SELECT 
        title, COUNT(rental_id) as cnt
    FROM
        film
        LEFT OUTER JOIN inventory USING (film_id)
        LEFT OUTER JOIN rental USING (inventory_id)
    GROUP BY
        film_id
    ORDER BY
        cnt ASC;
''')

Executando query:
Executando query:


Agora temos uma *view* chamada `movie_count`. Vamos verificar que ela funcionou, listando as 30 primeiras linhas:

In [34]:
db('''
SELECT * FROM movie_count LIMIT 30
''')

Executando query:
('ALICE FANTASIA', 0)
('APOLLO TEEN', 0)
('ARGONAUTS TOWN', 0)
('ARK RIDGEMONT', 0)
('ARSENIC INDEPENDENCE', 0)
('BOONDOCK BALLROOM', 0)
('BUTCH PANTHER', 0)
('CATCH AMISTAD', 0)
('CHINATOWN GLADIATOR', 0)
('CHOCOLATE DUCK', 0)
('COMMANDMENTS EXPRESS', 0)
('CROSSING DIVORCE', 0)
('CROWDS TELEMARK', 0)
('CRYSTAL BREAKING', 0)
('DAZED PUNK', 0)
('DELIVERANCE MULHOLLAND', 0)
('FIREHOUSE VIETNAM', 0)
('FLOATS GARDEN', 0)
('FRANKENSTEIN STRANGER', 0)
('GLADIATOR WESTWARD', 0)
('GUMP DATE', 0)
('HATE HANDICAP', 0)
('HOCUS FRIDA', 0)
('KENTUCKIAN GIANT', 0)
('KILL BROTHERHOOD', 0)
('MUPPET MILE', 0)
('ORDER BETRAYED', 0)
('PEARL DESTINY', 0)
('PERDITION FARGO', 0)
('PSYCHO SHRUNK', 0)


Agora suponha que alteramos a tabela `film`, mudando o nome do filme "DAZED PUNK" para "STONED PUNK".

**Atividade**: Do it.

In [35]:
db("""
UPDATE
    film
SET
    film.title = 'STONED PUNK'
WHERE 
    film.title = 'DAZED PUNK'
""")

Executando query:


Verifique agora a nossa *view*:

In [36]:
db('''
SELECT * FROM movie_count LIMIT 30
''')

Executando query:
('ALICE FANTASIA', 0)
('APOLLO TEEN', 0)
('ARGONAUTS TOWN', 0)
('ARK RIDGEMONT', 0)
('ARSENIC INDEPENDENCE', 0)
('BOONDOCK BALLROOM', 0)
('BUTCH PANTHER', 0)
('CATCH AMISTAD', 0)
('CHINATOWN GLADIATOR', 0)
('CHOCOLATE DUCK', 0)
('COMMANDMENTS EXPRESS', 0)
('CROSSING DIVORCE', 0)
('CROWDS TELEMARK', 0)
('CRYSTAL BREAKING', 0)
('STONED PUNK', 0)
('DELIVERANCE MULHOLLAND', 0)
('FIREHOUSE VIETNAM', 0)
('FLOATS GARDEN', 0)
('FRANKENSTEIN STRANGER', 0)
('GLADIATOR WESTWARD', 0)
('GUMP DATE', 0)
('HATE HANDICAP', 0)
('HOCUS FRIDA', 0)
('KENTUCKIAN GIANT', 0)
('KILL BROTHERHOOD', 0)
('MUPPET MILE', 0)
('ORDER BETRAYED', 0)
('PEARL DESTINY', 0)
('PERDITION FARGO', 0)
('PSYCHO SHRUNK', 0)


Como você pode ver, as views são tabelas virtuais que são automaticamente atualizadas quando as tabelas originais são modificadas.

In [37]:
connection.rollback()

## Tabelas temporárias

Tabelas temporárias podem ser criadas para ajudar nas tarefas de manipulação de dados. Essas tabelas existem apenas pela duração da sessão. Para criar uma tabela temporária, basta adicionar a palavra-chave `TEMPORARY` no momento da criação.

É comum criar tabelas temporárias à partir do resultado de comandos `SELECT`. Por exemplo, a seguinte query permite montar uma tabela temporária com os filmes que duram mais que 3 horas:

In [38]:
db('''
CREATE TEMPORARY TABLE long_film 
SELECT
    *
FROM
    film
WHERE
    film.length > 180;
''')

Executando query:


Podemos verificar que a tabela `long_film` agora existe:

In [39]:
db('DESCRIBE long_film')

Executando query:
('film_id', 'smallint unsigned', 'NO', '', '0', 'NULL')
('title', 'varchar(128)', 'NO', '', None, 'NULL')
('description', 'text', 'YES', '', None, 'NULL')
('release_year', 'year', 'YES', '', None, 'NULL')
('language_id', 'tinyint unsigned', 'NO', '', None, 'NULL')
('original_language_id', 'tinyint unsigned', 'YES', '', None, 'NULL')
('rental_duration', 'tinyint unsigned', 'NO', '', '3', 'NULL')
('rental_rate', 'decimal(4,2)', 'NO', '', '4.99', 'NULL')
('length', 'smallint unsigned', 'YES', '', None, 'NULL')
('replacement_cost', 'decimal(5,2)', 'NO', '', '19.99', 'NULL')
('rating', "enum('G','PG','PG-13','R','NC-17')", 'YES', '', 'G', 'NULL')
('special_features', "set('Trailers','Commentaries','Deleted Scenes','Behind the Scenes')", 'YES', '', None, 'NULL')
('last_update', 'timestamp', 'NO', '', 'CURRENT_TIMESTAMP', 'on update CURRENT_TIMESTAMP')


Muito embora ela não apareça na lista de tabelas: isso é um bug do MySQL. (https://dev.mysql.com/worklog/task/?id=648)

In [40]:
db('SHOW TABLES')

Executando query:
('actor',)
('actor_info',)
('address',)
('category',)
('city',)
('country',)
('customer',)
('customer_list',)
('film',)
('film_actor',)
('film_category',)
('film_list',)
('film_text',)
('inventory',)
('language',)
('movie_count',)
('nicer_but_slower_film_list',)
('payment',)
('rental',)
('sales_by_film_category',)
('sales_by_store',)
('staff',)
('staff_list',)
('store',)


Vamos listar o conteudo desta tabela:

In [41]:
db('SELECT title FROM long_film')

Executando query:
('ANALYZE HOOSIERS',)
('BAKED CLEOPATRA',)
('CATCH AMISTAD',)
('CHICAGO NORTH',)
('CONSPIRACY SPIRIT',)
('CONTROL ANTHEM',)
('CRYSTAL BREAKING',)
('DARN FORRESTER',)
('FRONTIER CABIN',)
('GANGS PRIDE',)
('HAUNTING PIANIST',)
('HOME PITY',)
('HOTEL HAPPINESS',)
('INTRIGUE WORST',)
('JACKET FRISCO',)
('KING EVOLUTION',)
('LAWLESS VISION',)
('LOVE SUICIDES',)
('MONSOON CAUSE',)
('MOONWALKER FOOL',)
('MUSCLE BRIGHT',)
('POND SEATTLE',)
('RECORDS ZORRO',)
('REDS POCUS',)
('RUNAWAY TENENBAUMS',)
('SATURN NAME',)
('SCALAWAG DUCK',)
('SEARCHERS WAIT',)
('SMOOCHY CONTROL',)
('SOLDIERS EVOLUTION',)
('SONS INTERVIEW',)
('SORORITY QUEEN',)
('STAR OPERATION',)
('SWEET BROTHERHOOD',)
('THEORY MERMAID',)
('WIFE TURN',)
('WILD APOLLO',)
('WORST BANGER',)
('YOUNG LANGUAGE',)


Vamos apagar a tabela `long_film`:

In [42]:
db('DROP TABLE long_film')

Executando query:


### Vamos praticar

- Crie uma tabela temporária `max_duration` que contém a duração máxima de filme para cada categoria

In [43]:
db("DROP TABLE IF EXISTS max_duration")
db("""
CREATE TEMPORARY TABLE max_duration 
SELECT
    category_id, category.name, MAX(film.length) as max_length
FROM
    film
    INNER JOIN film_category USING(film_id)
    INNER JOIN category USING(category_id)
GROUP BY
    category_id
""")

Executando query:
Executando query:


 - Verifique a tabela.

In [44]:
db("SELECT * FROM max_duration")

Executando query:
(1, 'Action', 185)
(2, 'Animation', 185)
(3, 'Children', 178)
(4, 'Classics', 184)
(5, 'Comedy', 185)
(6, 'Documentary', 183)
(7, 'Drama', 181)
(8, 'Family', 184)
(9, 'Foreign', 184)
(10, 'Games', 185)
(11, 'Horror', 181)
(12, 'Music', 185)
(13, 'New', 183)
(14, 'Sci-Fi', 185)
(15, 'Sports', 184)
(16, 'Travel', 185)


- Agora use a tabela temporária para construir uma tabela com as categorias e seus respectivos filmes mais longos:

In [45]:
db("""
SELECT
    film.title, max_duration.name 
FROM
    film
    INNER JOIN film_category USING(film_id)
    INNER JOIN max_duration USING(category_id)
WHERE
    film.length = max_duration.max_length
""")

Executando query:
('DARN FORRESTER', 'Action')
('WORST BANGER', 'Action')
('GANGS PRIDE', 'Animation')
('POND SEATTLE', 'Animation')
('FURY MURDER', 'Children')
('WRONG BEHAVIOR', 'Children')
('CONSPIRACY SPIRIT', 'Classics')
('CONTROL ANTHEM', 'Comedy')
('WIFE TURN', 'Documentary')
('YOUNG LANGUAGE', 'Documentary')
('JACKET FRISCO', 'Drama')
('KING EVOLUTION', 'Family')
('CRYSTAL BREAKING', 'Foreign')
('SORORITY QUEEN', 'Foreign')
('CHICAGO NORTH', 'Games')
('ANALYZE HOOSIERS', 'Horror')
('LOVE SUICIDES', 'Horror')
('HOME PITY', 'Music')
('FRONTIER CABIN', 'New')
('SOLDIERS EVOLUTION', 'Sci-Fi')
('SMOOCHY CONTROL', 'Sports')
('MUSCLE BRIGHT', 'Travel')
('SWEET BROTHERHOOD', 'Travel')


- delete a tabela temporária

In [46]:
db("DROP TABLE IF EXISTS max_duration")

Executando query:


### Desafio!

- Gere uma tabela contendo, para cada ator, a seguinte informação:

| first_name | last_name | filmes por categoria |
|--|--|--|
| PENELOPE | GUINESS | Animation: ANACONDA CONFESSIONS; Children: LANGUAGE COWBOY; Classics: COLOR PHILADELPHIA, WESTWARD SEABISCUIT; Comedy: VERTIGO NORTHWEST; Documentary: ACADEMY DINOSAUR; Family: KING EVOLUTION, SPLASH GUMP; Foreign: MULHOLLAND BEAST; Games: BULWORTH COMMANDMENTS, HUMAN GRAFFITI; Horror: ELEPHANT TROJAN, LADY STAGE, RULES HUMAN; Music: WIZARD COLDBLOODED; New: ANGELS LIFE, OKLAHOMA JUMANJI; Sci-Fi: CHEAPER CLYDE; Sports: GLEAMING JAWBREAKER |
| NICK | WAHLBERG | Action: BULL SHAWSHANK; Animation: FIGHT JAWBREAKER; Children: JERSEY SASSY; Classics: DRACULA CRYSTAL, GILBERT PELICAN; Comedy: MALLRATS UNITED, RUSHMORE MERMAID; Documentary: ADAPTATION HOLES; Drama: WARDROBE PHANTOM; Family: APACHE DIVINE, CHISUM BEHAVIOR, INDIAN LOVE, MAGUIRE APACHE; Foreign: BABY HALL, HAPPINESS UNITED; Games: ROOF CHAMPION; Music: LUCKY FLYING; New: DESTINY SATURDAY, FLASH WARS, JEKYLL FROGMEN, MASK PEACH; Sci-Fi: CHAINSAW UPTOWN, GOODFELLAS SALUTE; Travel: LIAISONS SWEET, SMILE EARRING |
| etc | etc | etc |

Dica: use `GROUP_CONCAT` para agrupar todas as strings de uma coluna em uma string só, e `CONCAT` para unir strings particulares.

In [47]:
db("""DROP TABLE IF EXISTS actor_category_films""")
db("""
CREATE TEMPORARY TABLE actor_category_films
    SELECT
        actor_id, first_name, last_name, CONCAT(name, ": ", GROUP_CONCAT(title)) as lista
    FROM
        actor
        INNER JOIN film_actor USING(actor_id)
        INNER JOIN film USING(film_id)
        INNER JOIN film_category USING(film_id)
        INNER JOIN category USING(category_id)
    GROUP BY
        actor_id, category_id
""")
db("""SELECT * FROM actor_category_films LIMIT 15""")

Executando query:
Executando query:
Executando query:
(1, 'PENELOPE', 'GUINESS', 'Animation: ANACONDA CONFESSIONS')
(1, 'PENELOPE', 'GUINESS', 'Children: LANGUAGE COWBOY')
(1, 'PENELOPE', 'GUINESS', 'Classics: COLOR PHILADELPHIA,WESTWARD SEABISCUIT')
(1, 'PENELOPE', 'GUINESS', 'Comedy: VERTIGO NORTHWEST')
(1, 'PENELOPE', 'GUINESS', 'Documentary: ACADEMY DINOSAUR')
(1, 'PENELOPE', 'GUINESS', 'Family: KING EVOLUTION,SPLASH GUMP')
(1, 'PENELOPE', 'GUINESS', 'Foreign: MULHOLLAND BEAST')
(1, 'PENELOPE', 'GUINESS', 'Games: BULWORTH COMMANDMENTS,HUMAN GRAFFITI')
(1, 'PENELOPE', 'GUINESS', 'Horror: ELEPHANT TROJAN,LADY STAGE,RULES HUMAN')
(1, 'PENELOPE', 'GUINESS', 'Music: WIZARD COLDBLOODED')
(1, 'PENELOPE', 'GUINESS', 'New: ANGELS LIFE,OKLAHOMA JUMANJI')
(1, 'PENELOPE', 'GUINESS', 'Sci-Fi: CHEAPER CLYDE')
(1, 'PENELOPE', 'GUINESS', 'Sports: GLEAMING JAWBREAKER')
(2, 'NICK', 'WAHLBERG', 'Action: BULL SHAWSHANK')
(2, 'NICK', 'WAHLBERG', 'Animation: FIGHT JAWBREAKER')


In [48]:
db("""
SET sql_mode=(SELECT REPLACE(@@sql_mode, 'ONLY_FULL_GROUP_BY', ''));
""")
db("""
SELECT
    first_name, last_name, GROUP_CONCAT(lista)
FROM
    actor_category_films
GROUP BY
    actor_id
LIMIT 3
""")

Executando query:
Executando query:
('PENELOPE', 'GUINESS', 'Animation: ANACONDA CONFESSIONS,Children: LANGUAGE COWBOY,Classics: COLOR PHILADELPHIA,WESTWARD SEABISCUIT,Comedy: VERTIGO NORTHWEST,Documentary: ACADEMY DINOSAUR,Family: KING EVOLUTION,SPLASH GUMP,Foreign: MULHOLLAND BEAST,Games: BULWORTH COMMANDMENTS,HUMAN GRAFFITI,Horror: ELEPHANT TROJAN,LADY STAGE,RULES HUMAN,Music: WIZARD COLDBLOODED,New: ANGELS LIFE,OKLAHOMA JUMANJI,Sci-Fi: CHEAPER CLYDE,Sports: GLEAMING JAWBREAKER')
('NICK', 'WAHLBERG', 'Action: BULL SHAWSHANK,Animation: FIGHT JAWBREAKER,Children: JERSEY SASSY,Classics: DRACULA CRYSTAL,GILBERT PELICAN,Comedy: MALLRATS UNITED,RUSHMORE MERMAID,Documentary: ADAPTATION HOLES,Drama: WARDROBE PHANTOM,Family: APACHE DIVINE,CHISUM BEHAVIOR,INDIAN LOVE,MAGUIRE APACHE,Foreign: BABY HALL,HAPPINESS UNITED,Games: ROOF CHAMPION,Music: LUCKY FLYING,New: DESTINY SATURDAY,FLASH WARS,JEKYLL FROGMEN,MASK PEACH,Sci-Fi: CHAINSAW UPTOWN,GOODFELLAS SALUTE,Travel: LIAISONS SWEET,SMILE EARRING

## Variáveis

Podemos montar uma query que retorne um valor só e armazenar este valor em uma variável, para uso posterior em outras queries. Para isso vamos usar o prefixo '@' para indicar variáveis, e o comando `SELECT ... INTO`.

Exemplo: quais são os filmes "caros" da nossa base sakila? Vamos descobrir quais filmes custam mais que um desvio padrão acima da média de preços de locação.

Primeiro vamos calcular a média e o desvio padrão dos preços de aluguel:

In [49]:
db('''
SELECT 
    AVG(rental_rate), 
    STDDEV(rental_rate)
INTO 
    @avg_rate, 
    @stddev_rate 
FROM
    film;
''')

Executando query:


Note que a query não retorna um resultado: o resultado foi armazenado direto nas variáveis `@avg_rate` e `@stddev_rate`. Vamos usar um `SELECT` sem tabelas para ver o resultado:

In [50]:
db('SELECT @avg_rate, @stddev_rate')

Executando query:
(Decimal('2.980000000'), 1.6455698101265719)


Agora podemos selecionar os filmes caros!

In [51]:
db('''
SELECT 
    title, rental_rate
FROM
    film
WHERE
    rental_rate > @avg_rate + @stddev_rate;
''')

Executando query:
('ACE GOLDFINGER', Decimal('4.99'))
('AIRPLANE SIERRA', Decimal('4.99'))
('AIRPORT POLLOCK', Decimal('4.99'))
('ALADDIN CALENDAR', Decimal('4.99'))
('ALI FOREVER', Decimal('4.99'))
('AMELIE HELLFIGHTERS', Decimal('4.99'))
('AMERICAN CIRCUS', Decimal('4.99'))
('ANTHEM LUKE', Decimal('4.99'))
('APACHE DIVINE', Decimal('4.99'))
('APOCALYPSE FLAMINGOS', Decimal('4.99'))
('ATTACKS HATE', Decimal('4.99'))
('ATTRACTION NEWTON', Decimal('4.99'))
('AUTUMN CROW', Decimal('4.99'))
('BABY HALL', Decimal('4.99'))
('BACKLASH UNDEFEATED', Decimal('4.99'))
('BEAST HUNCHBACK', Decimal('4.99'))
('BEAUTY GREASE', Decimal('4.99'))
('BEHAVIOR RUNAWAY', Decimal('4.99'))
('BETRAYED REAR', Decimal('4.99'))
('BIKINI BORROWERS', Decimal('4.99'))
('BILKO ANONYMOUS', Decimal('4.99'))
('BIRCH ANTITRUST', Decimal('4.99'))
('BIRD INDEPENDENCE', Decimal('4.99'))
('BIRDS PERDITION', Decimal('4.99'))
('BLINDNESS GUN', Decimal('4.99'))
('BOILED DARES', Decimal('4.99'))
('BOOGIE AMELIE', Decimal('4.99')

### Vamos praticar

Use variáveis temporárias para encontrar o ator que mais participou de filmes.

In [52]:
db("""
SELECT 
    actor.first_name, COUNT(actor_id) AS total
FROM
    actor
    INNER JOIN film_actor USING(actor_id)
GROUP BY
    actor_id
ORDER BY
    total DESC
LIMIT 10
""")

Executando query:
('GINA', 42)
('WALTER', 41)
('MARY', 40)
('MATTHEW', 39)
('SANDRA', 37)
('SCARLETT', 36)
('VAL', 35)
('VIVIEN', 35)
('GROUCHO', 35)
('UMA', 35)


Verifique quantas vezes o filme "COWBOY DOOM" foi alugado usando a view `movie_count`

In [53]:
db("""
SELECT
    *
FROM
    movie_count
WHERE
    title = 'COWBOY DOOM'
""")

Executando query:
('COWBOY DOOM', 7)


Registre um aluguel do filme "COWBOY DOOM", feito pelo funcionario "Jon Stephens" na loja id=1 para o cliente "JESSIE BANKS", na data '2019-01-01', com data de retorno '2019-01-08'

In [54]:
db("""SELECT film.film_id FROM film WHERE film.title = 'COWBOY DOOM'""")
db("""SELECT inventory.inventory_id FROM inventory WHERE inventory.film_id = 185 AND inventory.store_id = 1""")
db("""SELECT staff.staff_id FROM staff WHERE staff.first_name = 'Jon' AND staff.last_name = 'Stephens'""")
db("""SELECT customer.customer_id FROM customer WHERE customer.first_name = 'JESSIE' AND customer.last_name = 'BANKS'""")

db("""
INSERT INTO 
    rental (rental_date, inventory_id, customer_id, return_date, staff_id)
VALUES
    ('2019-01-01', 845, 215, '2019-01-08', '2')
""")

Executando query:
(185,)
Executando query:
(845,)
(846,)
Executando query:
(2,)
Executando query:
(215,)
Executando query:


Verifique usando a view que a contagem de alugueis do filme subiu.

In [55]:
db("""
SELECT
    *
FROM
    movie_count
WHERE
    title = 'COWBOY DOOM'
""")

Executando query:
('COWBOY DOOM', 8)


## Operador `IN`

Suponha que desejamos listar todos os filmes dos 3 atores mais populares. Podemos começar listando os 3 atores mais populares:

In [48]:
db('''
SELECT 
    actor_id, first_name, last_name, COUNT(film_id) AS num_films
FROM
    actor
    INNER JOIN film_actor USING (actor_id)
GROUP BY 
    actor_id
ORDER BY 
    num_films DESC
LIMIT 3
''')

Executando query:
(107, 'GINA', 'DEGENERES', 42)
(102, 'WALTER', 'TORN', 41)
(198, 'MARY', 'KEITEL', 40)


Vamos criar uma tabela temporária para guardar a informação de `actor_id` desses atores:

In [49]:
db('DROP TABLE IF EXISTS temp_pop_actors')
db('''
CREATE TEMPORARY TABLE temp_pop_actors
SELECT first_name, last_name, actor_id FROM
    actor
    INNER JOIN film_actor USING (actor_id)
GROUP BY 
    actor_id
ORDER BY 
    COUNT(film_id) DESC
LIMIT 3
''')
db('''
SELECT * from temp_pop_actors
''')

Executando query:
Executando query:
Executando query:
('GINA', 'DEGENERES', 107)
('WALTER', 'TORN', 102)
('MARY', 'KEITEL', 198)


Por fim, vamos usar essa informação para listar os filmes dos atores populares:

In [50]:
db('''
SELECT DISTINCT
    title
FROM
    film
    INNER JOIN film_actor USING (film_id)
WHERE
    actor_id IN (SELECT actor_id FROM temp_pop_actors);
''')

Executando query:
('BED HIGHBALL',)
('CALENDAR GUNFIGHT',)
('CHAMBER ITALIAN',)
('CHAPLIN LICENSE',)
('CHARIOTS CONSPIRACY',)
('CLUELESS BUCKET',)
('COLDBLOODED DARLING',)
('CONEHEADS SMOOCHY',)
('DARKNESS WAR',)
('DEER VIRGINIAN',)
('DOGMA FAMILY',)
('ELEPHANT TROJAN',)
('EXCITEMENT EVE',)
('FRISCO FORREST',)
('GANDHI KWAI',)
('GOODFELLAS SALUTE',)
('GUNFIGHT MOON',)
('HALL CASSIDY',)
('HEARTBREAKERS BRIGHT',)
('HOOK CHARIOTS',)
('HYDE DOCTOR',)
('IMPACT ALADDIN',)
('INDIAN LOVE',)
('INTRIGUE WORST',)
('LICENSE WEEKEND',)
('LOUISIANA HARRY',)
('MAGNIFICENT CHITTY',)
('METAL ARMAGEDDON',)
('MIDNIGHT WESTWARD',)
('MOVIE SHAKESPEARE',)
('MUMMY CREATURES',)
('OPEN AFRICAN',)
('SEARCHERS WAIT',)
('SEVEN SWARM',)
('SIERRA DIVIDE',)
('SPIRITED CASUALTIES',)
('STORM HAPPINESS',)
('SUGAR WONKA',)
('TELEGRAPH VOYAGE',)
('TRAINSPOTTING STRANGERS',)
('WIFE TURN',)
('WINDOW SIDE',)
('AMELIE HELLFIGHTERS',)
('ARABIA DOGMA',)
('BANG KWAI',)
('CASABLANCA SUPER',)
('CASPER DRAGONFLY',)
('CROW GREASE',

Note o uso de *subqueries*!

Não se esqueça de limpar tudo no final!

In [51]:
db('DROP TABLE temp_pop_actors')

Executando query:


### Vamos praticar

Liste os atores que participaram dos 3 filmes mais rentáveis (aqueles que mais geraram receita para a locadora).

## Subqueries

Os tópicos discutidos acima poderiam ser resolvidos, em grande parte, com subqueries. As subqueries são queries `SELECT` criadas dentro de outras queries. 

Poderíamos ter usado subqueries nos mesmos lugares onde usamos tabelas temporárias, nos tópicos acima. Quando a subquery pode ser transformada em uma tabela temporária independente, separada da query exterior, dizemos que a subquery é **não-correlacionada** com a query exterior.

Usar subqueries não-correlacionadas é um tópico controverso: podemos sempre usar uma tabela temporária ou, ás vezes, pensar em um `JOIN` simples. Aliás, muitas vezes o otimizador de queries do banco de dados transformará a subquery em `JOIN`, se isso for vantajoso em termos de desempenho.

Uma subquery que depende da query externa (e portanto não pode ser separada em uma tabela temporária independente) é chamada de **subquery correlacionada**. Nestes casos podemos ter que executar a subquery para cada linha da query exterior! 

### Vamos praticar

Vamos refazer a atividade dos filmes de atores populares, usando subqueries. 

Temos um problema: o MySQL não suporta ``LIMIT`` em subqueries com o operador ``IN``. Vamos investigar isso mais de perto. 

Em primeiro lugar faça uma tradução direta da implementação da atividade anterior trocando tabela temporária por subquery.

In [52]:
try:
    db('''
    SELECT 
        actor_id, first_name, last_name
    FROM
        actor
        INNER JOIN film_actor USING (actor_id)
    WHERE
        film_actor.film_id IN (
            SELECT 
                film_id
            FROM
                film
                INNER JOIN inventory USING (film_id)
                INNER JOIN rental USING (inventory_id)
                INNER JOIN payment USING (rental_id)
            WHERE
                payment_date IS NOT NULL
            GROUP BY film_id
            ORDER BY SUM(amount)
            LIMIT 3)
    ORDER BY actor_id
    ''')
except mysql.connector.ProgrammingError as e:
    print(f'ProgrammingError: {e}')

Executando query:
ProgrammingError: 1235 (42000): This version of MySQL doesn't yet support 'LIMIT & IN/ALL/ANY/SOME subquery'


Ok, apareceu o problema. Mas considere que o problema original não precisava de IN desde o começo! Construa essa solução.

# `UNION`

Quando duas tabelas tem **EXATAMENTE** as mesmas colunas, podemos concatená-las e formar uma grande tabela unificada usando o operador `UNION`. Por exemplo: suponha que desejamos montar uma lista dos nomes e sobrenomes de todos os clientes E de todos os funcionários. Eis uma solução possível:

In [53]:
db('DROP TABLE IF EXISTS nomes_clientes')
db('CREATE TEMPORARY TABLE nomes_clientes SELECT first_name, last_name FROM customer')

Executando query:
Executando query:


In [54]:
db('DESCRIBE nomes_clientes')
db('SELECT * FROM nomes_clientes LIMIT 5')

Executando query:
('first_name', 'varchar(45)', 'NO', '', None, 'NULL')
('last_name', 'varchar(45)', 'NO', '', None, 'NULL')
Executando query:
('MARY', 'SMITH')
('PATRICIA', 'JOHNSON')
('LINDA', 'WILLIAMS')
('BARBARA', 'JONES')
('ELIZABETH', 'BROWN')


In [55]:
db('DROP TABLE IF EXISTS nomes_staff')
db('CREATE TEMPORARY TABLE nomes_staff SELECT first_name, last_name FROM staff')

Executando query:
Executando query:


In [56]:
db('DESCRIBE nomes_staff')
db('SELECT * FROM nomes_staff LIMIT 5')

Executando query:
('first_name', 'varchar(45)', 'NO', '', None, 'NULL')
('last_name', 'varchar(45)', 'NO', '', None, 'NULL')
Executando query:
('Mike', 'Hillyer')
('Jon', 'Stephens')


In [57]:
db('DROP TABLE IF EXISTS nomes_all')
db('CREATE TEMPORARY TABLE nomes_all (SELECT * FROM nomes_staff) UNION (SELECT * FROM nomes_clientes)')

Executando query:
Executando query:


In [58]:
db('DESCRIBE nomes_all')
db('SELECT * FROM nomes_all LIMIT 5')

Executando query:
('first_name', 'varchar(45)', 'NO', '', '', 'NULL')
('last_name', 'varchar(45)', 'NO', '', '', 'NULL')
Executando query:
('Mike', 'Hillyer')
('Jon', 'Stephens')
('MARY', 'SMITH')
('PATRICIA', 'JOHNSON')
('LINDA', 'WILLIAMS')


In [59]:
db('DROP TABLE nomes_clientes')
db('DROP TABLE nomes_staff')
db('DROP TABLE nomes_all')

Executando query:
Executando query:
Executando query:


**Vamos praticar:** refaça o exemplo acima mas use *subqueries* ao invés de *temp tables*.

## Desafios!

Faça uma lista de filmes que tenham mais de dois atores cujo nome inicia com a mesma letra do título do filme!

Semana do "DAN HARRIS": liste os clientes que nunca assistiram um filme do ator "DAN HARRIS" ou que já assistiram mas onde a ultima vez em que assistiram um filme dele foi antes de '2005-06-01'

## Conclusão

Façamos uma pausa para apreciar quão longe estamos: já conseguimos criar nossas tabelas, inserir informação, removê-la, atualizá-la, e consultar nossa base de maneiras bem sofisticadas! Vimos desde `SELECT` simples até buscas mais complexas envolvendo várias etapas de processamento para obter o dado desejado.

Por hoje é só, feche sua conexão e bom descanso!

In [29]:
connection.close()